# Training ResNet with NSL

In this notebook I will train a Flower Classification model using the pretrained ResNet model.

In [1]:
!nvidia-smi

Mon Jul  5 19:23:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Importing Libraries

In [2]:
!pip install neural-structured-learning --quiet
!pip install tensorflow_addons --quiet

In [3]:
import os
import glob
import random
import warnings
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_addons as tfa
import neural_structured_learning as nsl

from functools import partial
from sklearn.metrics import roc_auc_score

warnings.simplefilter('ignore')

## Configuration

In [19]:
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed_everything(1234)

In [20]:
class Config:
  
  INPUT_DIR = '/content/drive/MyDrive/Projects/Flower_Classification/input'
  MODEL_DIR = '/content/drive/MyDrive/Projects/Flower_Classification/models'
  LOG_DIR = '/content/drive/MyDrive/Projects/Flower_Classification/log_nsl'

  AUTOTUNE = tf.data.experimental.AUTOTUNE
  
  IMAGE_SIZE = 224
  IMAGE_HEIGHT = 224
  IMAGE_WIDTH = 224
  TRAIN_BATCH_SIZE = 8
  VALID_BATCH_SIZE = 8
  SHUFFLE = 1234

  LR = 0.001
  WEIGHT_DECAY = 0.0001
  NUM_CLASSES = 104
  WARMUP_EPOCHS = 10

  FINETUNE_EPOCHS = 30
  FINETUNE_LR = 0.0001

  ADV_MULTIPLIER = 0.2
  ADV_STEP_SIZE = 0.05

In [21]:
config = Config()

## Creating the dataset

In [7]:
def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.reshape(image, [config.IMAGE_HEIGHT, config.IMAGE_WIDTH, 3])
    return image

In [8]:
def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.float32)
    return image, label

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

In [9]:
def load_dataset(filenames, labeled=True, ordered=False):
  ignore_order = tf.data.Options()
  if not ordered:
    ignore_order.experimental_deterministic = False
  dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=config.AUTOTUNE)
  dataset = dataset.with_options(ignore_order)
  dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=config.AUTOTUNE)
  return dataset

In [10]:
def train_data_augment(image, label):
    crop_size = tf.random.uniform([], int(config.IMAGE_HEIGHT*.7), config.IMAGE_HEIGHT, dtype=tf.int32)
        
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_saturation(image, lower=0, upper=2)
    image = tf.image.random_crop(image, size=[crop_size, crop_size, 3])
    image = tf.image.resize(image, size=[config.IMAGE_HEIGHT, config.IMAGE_WIDTH])

    return image, label

In [11]:
# Train Dataset
train_filenames = glob.glob('/content/drive/MyDrive/Projects/Flower_Classification/input/tfrecords-jpeg-224x224/train/*.tfr*')

train_dataset = load_dataset(train_filenames)
train_dataset = train_dataset.map(train_data_augment, num_parallel_calls=config.AUTOTUNE)
train_dataset = train_dataset.repeat()
train_dataset = train_dataset.shuffle(config.SHUFFLE)
train_dataset = train_dataset.batch(config.TRAIN_BATCH_SIZE)
train_dataset = train_dataset.prefetch(config.AUTOTUNE)

In [12]:
# Validation Dataset
valid_filenames = glob.glob('/content/drive/MyDrive/Projects/Flower_Classification/input/tfrecords-jpeg-224x224/val/*.tfr*')

valid_dataset = load_dataset(valid_filenames)
valid_dataset = valid_dataset.batch(config.TRAIN_BATCH_SIZE)
valid_dataset = valid_dataset.cache()
valid_dataset = valid_dataset.prefetch(config.AUTOTUNE)

## Creating the Model

In [13]:
base_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/resnet_50/feature_vector/1", trainable=False)

In [14]:
model = tf.keras.Sequential([
                             tf.keras.layers.InputLayer(input_shape=[config.IMAGE_HEIGHT, config.IMAGE_WIDTH, 3]),
                             base_layer,
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Dropout(0.3),
                             tf.keras.layers.Dense(config.NUM_CLASSES, activation='softmax')
])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 2048)              23561152  
_________________________________________________________________
batch_normalization (BatchNo (None, 2048)              8192      
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 104)               213096    
Total params: 23,782,440
Trainable params: 217,192
Non-trainable params: 23,565,248
_________________________________________________________________


## Training the Model

In [16]:
optimizer = tfa.optimizers.AdamW(learning_rate=config.LR, weight_decay=config.WEIGHT_DECAY)
loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [17]:
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(config.MODEL_DIR + '/ResNet', save_best_only=True, mode='min', save_weights_only=False)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=3)
logger = tf.keras.callbacks.TensorBoard(log_dir=config.LOG_DIR)
lr_schedular = tf.keras.callbacks.ReduceLROnPlateau(patience=1)

In [23]:
model.compile(optimizer=optimizer, loss=loss, metrics=['sparse_categorical_accuracy'])

model.fit(x = train_dataset,
          validation_data = valid_dataset,
          epochs = config.WARMUP_EPOCHS,
          steps_per_epoch=2700,
          callbacks=[early_stopping, logger, lr_schedular])

Epoch 1/5
2700/2700 [==============================] - 60s 21ms/step - loss: 0.8690 - sparse_categorical_accuracy: 0.7647 - val_loss: 0.7366 - val_sparse_categorical_accuracy: 0.8163
Epoch 2/5
2700/2700 [==============================] - 56s 21ms/step - loss: 0.8666 - sparse_categorical_accuracy: 0.7637 - val_loss: 0.7560 - val_sparse_categorical_accuracy: 0.8184
Epoch 3/5
2700/2700 [==============================] - 56s 21ms/step - loss: 0.8836 - sparse_categorical_accuracy: 0.7921 - val_loss: 0.8706 - val_sparse_categorical_accuracy: 0.8260
Epoch 4/5
2700/2700 [==============================] - 56s 21ms/step - loss: 1.2801 - sparse_categorical_accuracy: 0.7931 - val_loss: 1.4049 - val_sparse_categorical_accuracy: 0.8147


In [24]:
model.save(config.MODEL_DIR + '/ResNet_trained')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Projects/Flower_Classification/models/ResNet_trained/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Projects/Flower_Classification/models/ResNet_trained/assets


## Finetuning the Whole Model

In [25]:
model = tf.keras.models.load_model(config.MODEL_DIR + '/ResNet_trained')

In [30]:
for layer in model.layers:
  layer.trainable = True

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 2048)              23561152  
_________________________________________________________________
batch_normalization (BatchNo (None, 2048)              8192      
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 104)               213096    
Total params: 23,782,440
Trainable params: 23,725,224
Non-trainable params: 57,216
_________________________________________________________________


In [27]:
# checking the loaded model
model.evaluate(valid_dataset)

464/464 [==============================] - 7s 13ms/step - loss: 1.4049 - sparse_categorical_accuracy: 0.8147


[1.4048511981964111, 0.8146551847457886]

In [28]:
optimizer = tf.keras.optimizers.Adam(learning_rate=config.FINETUNE_LR)
loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [29]:
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(config.MODEL_DIR + '/ResNet_finetuning.h5', save_best_only=True, mode='min', save_weights_only=False)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=3)
logger = tf.keras.callbacks.TensorBoard(log_dir=config.LOG_DIR)
lr_schedular = tf.keras.callbacks.ReduceLROnPlateau(patience=1, min_delta=0.01)

In [31]:
model.compile(optimizer=optimizer, loss=loss, metrics=['sparse_categorical_accuracy'])

model.fit(x = train_dataset,
          validation_data = valid_dataset,
          epochs = config.FINETUNE_EPOCHS,
          steps_per_epoch = 2700,
          callbacks=[early_stopping, logger, lr_schedular])

Epoch 1/30
2700/2700 [==============================] - 188s 66ms/step - loss: 1.1320 - sparse_categorical_accuracy: 0.7311 - val_loss: 0.9467 - val_sparse_categorical_accuracy: 0.7527
Epoch 2/30
2700/2700 [==============================] - 176s 65ms/step - loss: 0.7476 - sparse_categorical_accuracy: 0.8092 - val_loss: 0.7423 - val_sparse_categorical_accuracy: 0.8122
Epoch 3/30
2700/2700 [==============================] - 177s 66ms/step - loss: 0.6126 - sparse_categorical_accuracy: 0.8400 - val_loss: 0.6939 - val_sparse_categorical_accuracy: 0.8265
Epoch 4/30
2700/2700 [==============================] - 176s 65ms/step - loss: 0.5335 - sparse_categorical_accuracy: 0.8563 - val_loss: 0.6591 - val_sparse_categorical_accuracy: 0.8249
Epoch 5/30
2700/2700 [==============================] - 178s 66ms/step - loss: 0.4795 - sparse_categorical_accuracy: 0.8679 - val_loss: 0.5361 - val_sparse_categorical_accuracy: 0.8561
Epoch 6/30
2700/2700 [==============================] - 181s 67ms/step - lo

In [32]:
model.save(config.MODEL_DIR + '/ResNet_finetuned')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Projects/Flower_Classification/models/ResNet_finetuned/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Projects/Flower_Classification/models/ResNet_finetuned/assets


## Conclusion
As we can observer our model is overfitting. I have already introduced a 30% drouput in the model but further regularizing the model could help but for now we will stay put at this. Our final validation accuray is 0.9106 as we have saved the model with least loss.